### TextEmbeddings
- 교재 140쪽 ~ 149쪽 내용

#### 환경설정

In [1]:
!pip install --quiet langchain

In [2]:
# 01 colab용 비밀 키 불러오기
from google.colab import userdata
import os
from openai import OpenAI # openai-python SDK

In [3]:
# 02 OpenAI API키 설정
api_key = userdata.get("modulabsKey") # colab 사용자 환경변수 탭에서 등록한 키
os.environ["OPEN_API_KEY"] = api_key
client = OpenAI(api_key = api_key)

In [4]:
# 03 버전 확인(선택)
import openai
import langchain
print("LangChain ver.",langchain.__version__)
print("OpenAI SDK ver.", openai.__version__)

LangChain ver. 0.1.17
OpenAI SDK ver. 1.91.0


#### 01 OpenAI의 text-embedding-3-small모델
- Closed source 임베딩 모델 중 하나

In [5]:
# !pip install --quiet langchain_openai

In [6]:
# 설치 후 런타임 재시작
!pip install --quiet \
  langchain==0.0.353 \
  langchain-openai==0.0.6 \
  openai==0.28.1 \
  pydantic<2

/bin/bash: line 1: 2: No such file or directory


In [7]:
import os
from langchain_openai import OpenAIEmbeddings
api_key = userdata.get("modulabsKey")
os.environ["OPENAI_API_KEY"] = api_key

embeddings_model = OpenAIEmbeddings(model = 'text-embedding-3-small')
embeddings = embeddings_model.embed_documents(
    [
     "Hi there!",
     "Oh, hello!",
     "What's your name?",
     "My friends call me World",
     "Hello world"
    ]
)

len(embeddings), len(embeddings[0])

(5, 1536)

In [9]:
# langchain_community 설치
!pip install --quiet langchain-community

In [10]:
# colab에서 drive mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
!pip install --quiet pypdfium2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 28.5 MB/s eta 0:00:00


In [15]:
from langchain_community.document_loaders import PyPDFium2Loader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 임베딩 모델 API 호출
embeddings_model = OpenAIEmbeddings(model = 'text-embedding-3-small')

# PDF 문서 로드
loader = PyPDFium2Loader(r"/content/drive/MyDrive/ColabNotebooks/03_Modulabs/Modu_LLM/data/[이슈리포트 2022-2호] 혁신성장 정책금융 동향.pdf")
pages = loader.load()

# PDF 문서를 여러 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 100
)

texts = text_splitter.split_documents(pages)

# OpenAI 임베딩 모델로 청크들을 임베딩 변환하기
embeddings = embeddings_model.embed_documents([i.page_content for i in texts])

len(embeddings), len(embeddings[0])

(53, 1536)

In [16]:
examples = embeddings_model.embed_documents(
    [
        "안녕하세요",
        "제 이름은 홍두깨입니다.",
        "이름이 무엇인가요?",
        "랭체인은 유용합니다.",
    ]
)

# 예시 질문과 답변 임베딩
embedded_query_q = embeddings_model.embed_query("이 대화에서 언급된 이름은 무엇입니까?")
embedded_query_a = embeddings_model.embed_query("이 대화에서 언급된 이름은 홍길동입니다.")

In [20]:
from numpy import dot
from numpy.linalg import norm
import numpy as np

def cos_sim(A, B):
  return dot(A,B)/(norm(A) * norm(B))

print(cos_sim(embedded_query_q, embedded_query_a))
print(cos_sim(embedded_query_a, examples[1]))
print(cos_sim(embedded_query_a, examples[3]))

0.6491181289078306
0.47019079309112843
0.148439509966488


#### 02 jhgan/ko-sroberta-multitask 임베딩
- Open source 임베딩 모델 활용

In [24]:
!pip install --quiet sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 831.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.0 MB/s eta 0:00:00


In [25]:
from langchain_community.embeddings import HuggingFaceEmbeddings

# HuggingfaceEmbedding 함수로 Open source 임베딩 모델 로드
model_name = "jhgan/ko-sroberta-multitask"
ko_embedding = HuggingFaceEmbeddings(
    model_name = model_name
)

examples = ko_embedding.embed_documents(
    [
        "안녕하세요",
        "제 이름은 홍두깨입니다.",
        "이름이 무엇인가요?",
        "랭체인은 유용합니다.",
    ]
)

embedded_query_q = ko_embedding.embed_query("이 대화에서 언급된 이름은 무엇입니까?")
embedded_query_a = ko_embedding.embed_query("이 대화에서 언급된 이름은 홍길동입니다.")

print(cos_sim(embedded_query_q, embedded_query_a))
print(cos_sim(embedded_query_q, examples[1]))
print(cos_sim(embedded_query_q, examples[3]))

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

0.6070005852394463
0.2947341657162066
0.2757840706251745


#### 03 BAAI/bge-small-en 임베딩

In [26]:
from langchain_community.embeddings import HuggingFaceEmbeddings

model_name = "BAAI/bge-small-en"
bge_embedding = HuggingFaceEmbeddings(
    model_name = model_name
)

examples = bge_embedding.embed_documents(
    [
        "안녕하세요",
        "제 이름은 홍두깨입니다.",
        "이름이 무엇인가요?",
        "랭체인은 유용합니다.",
    ]
)

embedded_query_q = bge_embedding.embed_query("이 대화에서 언급된 이름은 무엇입니까?")
embedded_query_a = bge_embedding.embed_query("이 대화에서 언급된 이름은 홍길동입니다.")

print(cos_sim(embedded_query_q, embedded_query_a))
print(cos_sim(embedded_query_q, examples[1]))
print(cos_sim(embedded_query_q, examples[3]))

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

0.955454149782215
0.9431682731233773
0.8853417113547791
